In [1]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta
import random
from copy import deepcopy
from django.contrib.gis.geos import LineString, Point
from django.contrib.admin.utils import flatten
import math
import datetime
import locale
from tqdm.notebook import tqdm
from PIL import Image
from io import BytesIO
from django.contrib.gis.geos import Point, LineString as DjangoLineString
import pytz

In [2]:
from simple_history.utils import bulk_create_with_history
from helpers.histories import bulk_update_with_history
from openpyxl_image_loader import SheetImageLoader

In [3]:
from helpers.route_maker import dic_to_ordered_list, unequal_point_pairs

In [4]:
def km_to_coordinates(road, km):

    road_marks = dic_to_ordered_list(road.marks)
    selected_pair = None

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] > km and pair[1]["km"] < km:
            selected_pair = pair
            break
        elif pair[1]["km"] > km and pair[0]["km"] < km:
            selected_pair = pair
            break
        elif pair[0]["km"] == km:
            return Point(pair[0]["point"]["coordinates"]), road
        elif pair[1]["km"] == km:
            return Point(pair[1]["point"]["coordinates"]), road

    # Calculate total length
    start_km = min([selected_pair[0]["km"], selected_pair[1]["km"]])
    end_km = max([selected_pair[0]["km"], selected_pair[1]["km"]])
    segment_mark_length = end_km - start_km
    distance_from_min = km - start_km
    # Check if km is decreasing
    invert_km = selected_pair[1]["km"] <= selected_pair[0]["km"]

    # Cut segment
    start_key = min((selected_pair[0]["index"], selected_pair[1]["index"]))
    end_key = max((selected_pair[0]["index"], selected_pair[1]["index"]))

    points = []
    for key in range(start_key, end_key + 1):
        points.append(
            Point(road.path[key][0], road.path[key][1], road.path[key][2])
        )

    segment = LineString(points, srid=4326)
    # Apply km inversion correction
    if invert_km:
        segment_distance = (
            1 - (distance_from_min / segment_mark_length)
        ) * segment.length
    else:
        segment_distance = (
            distance_from_min / segment_mark_length
        ) * segment.length
    # Find point
    point = segment.interpolate(segment_distance)

    return Point(point.x, point.y), road

In [5]:
def check_valid_road(road, km):

    road_marks = dic_to_ordered_list(road.marks)

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] >= km and pair[1]["km"] <= km:
            return True
        elif pair[1]["km"] >= km and pair[0]["km"] <= km:
            return True

    return False

In [6]:
def get_road_coordinates(road_name, km, direction, company):
    road_set = Road.objects.filter(
        name__contains=road_name, direction=int(direction), company=company
    )
#     print(road_set)

    # If roads in specified direction are not found, search road only
    # by name and order then by direction
    road_set_generic = Road.objects.filter(
        name__contains=road_name, company=company
    ).order_by("direction")

    # Check if KM range in road_set
#     print("Searching KM on direction...")
    valid = False
    for road in road_set:
        if check_valid_road(road, km):
#             print("Found KM on Road {}".format(road))
            valid = True
            break

    if not valid:
#         print("Searching KM without direction...")
        for road in road_set_generic:
            if check_valid_road(road, km):
#                 print("Found KM on Road {}".format(road))
                valid = True
                break

    if not valid:
        return Point(0, 0), None

    try:
        return km_to_coordinates(road, km)
    except Exception:
        return Point(0, 0), None

In [7]:
def get_connected_reference(company, resource, key, value):
    value=value.strip()
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    return next(a['value'] for a in select_options if a['name'].lower() == value.lower())

In [8]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)
    if field == 'Nome da placa':
        value=value.upper()
    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

In [9]:
def get_sign(field, sign_field, occ, value):
    if sign_field == 'Cor':
        value=value.capitalize()
    elif sign_field == 'Tipo de Película':
        value=value.split(' ')[1]
    values = occ.form_fields['fields']
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

In [10]:
!cat .env

STAGE=PRODUCTION


In [11]:
# filename = 'acessos_importacao_v2'
filename='Acessos - SP 330 - 240+500 ao 318+000.r5' # 16/05/23

In [12]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[1]
# sheetname
ws = wb[sheetname]
ws

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


<Worksheet "Acessos">

In [13]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
        
    values.append(obj)

In [14]:
company = Company.objects.get(name='Arteris Via Paulista - Faixa de Domínio')

occurrence_type=OccurrenceType.objects.get(uuid='7c44852b-84d8-453f-bfc9-d3e2e92602b2')
equipe=Firm.objects.get(name="Coordenadores Faixa", company=company)
status=ServiceOrderActionStatus.objects.get(name='Resolvido',companies=company)
user=User.objects.get(username='rlcs')
utc = pytz.timezone('America/Sao_Paulo')
company,occurrence_type,equipe,status

(<Company: ec364a8e-b335-4729-8ac8-50ccabd73459: Arteris Via Paulista - Faixa de Domínio>,
 <OccurrenceType: Acessos - ['Arteris Via Paulista - Faixa de Domínio']>,
 <Firm: [ec364a8e-b335-4729-8ac8-50ccabd73459: Arteris Via Paulista - Faixa de Domínio] 5d9b180c-22d3-4e8a-a887-de3cfe7c4cc9: Coordenadores Faixa>,
 <ServiceOrderActionStatus: Resolvido - REPORTING_STATUS>)

In [15]:
len(values),values[0]

(232,
 {'km': 240.56,
  'km final': None,
  'km de Projeto': None,
  'km final de Projeto': None,
  'Status': 'Resolvido',
  'Equipe/Empreiteira': 'Coordenadores Faixa',
  'Encontrado em': None,
  'Executado em': None,
  'Prazo': None,
  'Sentido': 'Sul',
  'Classe': 'Acessos',
  'Faixa': 'Faixa 1',
  'Pista': 'Tronco Dupla',
  'km de referência': None,
  'Ramo': None,
  'Rodovia': 'SP-330',
  'Código Do Acesso': 'SP330 - 01 - S',
  'Código Novo': '029-C-SP330-240-S01',
  'Lote': 29,
  'Tipo Da Obra': 'Concessão SP 330',
  'Localização': 240.56,
  'Latitude': -21.75879,
  'Longitude': -47.560935,
  'Cidade': 'Santa Rita do Passa Quatro',
  'Interessado': 'Roberto Missiatto',
  'Matrícula': 17573,
  'Cri': 'CRI Santa Rita do Passa Quatro ',
  'Tipo Do Acesso': 'Simples',
  'Classificação': 'Comercial',
  'Atividade Do Acesso': 'Particular',
  'Atividade': 'D01',
  'Autorizado?': 'NÃO',
  'Nº Protocolo Artesp': None,
  'Nº Processo Artesp': None,
  'Termo De Autorização': 'Não Autorizado

In [16]:
for a in tqdm(values):
    a['Data Notificações_1'] = None if a['Data Notificações_1'] == '' or  a['Data Notificações_1'] == ' ' or  a['Data Notificações_1'] == '  ' else a['Data Notificações_1']
    a['Data Notificações_2'] = None if a['Data Notificações_2'] == '' or  a['Data Notificações_2'] == ' ' or  a['Data Notificações_2'] == '  ' else a['Data Notificações_2']
    a['Data Notificações_3'] = None if a['Data Notificações_3'] == '' or  a['Data Notificações_3'] == ' ' or  a['Data Notificações_3'] == '  ' else a['Data Notificações_3']
    a['Notificacao'] = list(filter(None, [{'letter_number_notification_step': a['Número Carta Notificações_1'], 'data_notification_step': a['Data Notificações_1'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Data Notificações_1'] != ' ' and a['Data Notificações_1'] != '  ' else None, 'ar_notification_step': a['Número Protocolo Notificações_1'], 'recebido_por_notification_step': str(a['Recebido Por Notificações_1'])} if a['Data Notificações_1'] is not None and a['Data Notificações_1'] != ' ' and a['Data Notificações_1'] != '  ' else None,
                                  {'letter_number_notification_step': a['Número Carta Notificações_2'], 'data_notification_step': a['Data Notificações_2'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Data Notificações_2'] != ' ' and a['Data Notificações_2'] != '  ' else None, 'ar_notification_step': a['Número Protocolo Notificações_2'], 'recebido_por_notification_step': str(a['Recebido Por Notificações_2'])} if a['Data Notificações_2'] is not None and a['Data Notificações_2'] != ' ' and a['Data Notificações_2'] != '  '  else None,
                                  {'letter_number_notification_step': a['Número Carta Notificações_3'], 'data_notification_step': a['Data Notificações_3'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Data Notificações_3'] != ' ' and a['Data Notificações_3'] != '  ' else None, 'ar_notification_step': a['Número Protocolo Notificações_3'], 'recebido_por_notification_step': str(a['Recebido Por Notificações_3'])} if a['Data Notificações_3'] is not None and a['Data Notificações_3'] != ' ' and a['Data Notificações_3'] != '  '  else None]))

  0%|          | 0/232 [00:00<?, ?it/s]

In [17]:
for a in tqdm(values):
    a['Data Termo de ciência_1'] = None if a['Data Termo de ciência_1'] == '' or a['Data Termo de ciência_1'] == ' ' or a['Data Termo de ciência_1'] == '  ' or a['Data Termo de ciência_1'] == '-' else a['Data Termo de ciência_1']
    a['Data Termo de ciência_2'] = None if a['Data Termo de ciência_2'] == '' or a['Data Termo de ciência_2'] == ' ' or a['Data Termo de ciência_2'] == '  ' or a['Data Termo de ciência_2'] == '-' else a['Data Termo de ciência_2']
    a['Data Termo de ciência_3'] = None if a['Data Termo de ciência_3'] == '' or a['Data Termo de ciência_3'] == ' ' or a['Data Termo de ciência_3'] == '  ' or a['Data Termo de ciência_3'] == '-' else a['Data Termo de ciência_3']
    a['Termo Ciencia'] = list(filter(None, [{'letter_number_cience_step': a['Número Carta Termo de ciência_1'], 'data_cience_step': a['Data Termo de ciência_1'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Data Termo de ciência_1'] != ' ' and a['Data Termo de ciência_1'] != '  ' else None, 'ar_cience_step': a['Número Protocolo Termo de ciência_1'], 'recebido_por_cience_step': a['Recebido Por Termo de ciência_1']} if a['Data Termo de ciência_1'] is not None and a['Data Termo de ciência_1'] != ' ' and a['Data Termo de ciência_1'] != '  ' else None,
                                  {'letter_number_cience_step': a['Número Carta Termo de ciência_2'], 'data_cience_step': a['Data Termo de ciência_2'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Data Termo de ciência_2'] != ' ' and a['Data Termo de ciência_2'] != '  ' else None, 'ar_cience_step': a['Número Protocolo Termo de ciência_2'], 'recebido_por_cience_step': a['Recebido Por Termo de ciência_2']} if a['Data Termo de ciência_2'] is not None and a['Data Termo de ciência_2'] != ' ' and a['Data Termo de ciência_2'] != '  ' else None,
                                  {'letter_number_cience_step': a['Número Carta Termo de ciência_3'], 'data_cience_step': a['Data Termo de ciência_3'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Data Termo de ciência_3'] != ' ' and a['Data Termo de ciência_3'] != '  ' else None, 'ar_cience_step': a['Número Protocolo Termo de ciência_3'], 'recebido_por_cience_step': a['Recebido Por Termo de ciência_3']} if a['Data Termo de ciência_3'] is not None  and a['Data Termo de ciência_3'] != ' ' and a['Data Termo de ciência_3'] != '  ' else None]))

  0%|          | 0/232 [00:00<?, ?it/s]

In [18]:
for a in tqdm(values):
    a['Termo Anuencia'] = list(filter(None, [{'letter_number_anuence_step': a['Número Carta Termo de anuência_1'], 'data_anuence_step': a['Data Termo de anuência_1'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Data Termo de anuência_1'] and a['Data Termo de anuência_1'] != ' ' else None, 'ar_anuence_step': a['Número Protocolo Termo de anuência_1'], 'recebido_por_anuence_step': a['Recebido Por Termo de anuência_1']} if a['Data Termo de anuência_1'] is not None and a['Data Termo de anuência_1'] != ' ' and a['Data Termo de anuência_1'] != '  ' else None,
                                  {'letter_number_anuence_step': a['Número Carta Termo de anuência_2'], 'data_anuence_step': a['Data Termo de anuência_2'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Data Termo de anuência_2'] and a['Data Termo de anuência_2'] != ' ' else None, 'ar_anuence_step': a['Número Protocolo Termo de anuência_2'], 'recebido_por_anuence_step': a['Recebido Por Termo de anuência_2']} if a['Data Termo de anuência_2'] is not None and a['Data Termo de anuência_2'] != ' ' and a['Data Termo de anuência_2'] != '  ' else None]))

  0%|          | 0/232 [00:00<?, ?it/s]

In [23]:
for a in values:
    if a['Procedimento'] is not None:
        a['Procedimento'] = a['Procedimento'].capitalize()
    if a.get('Status Projeto D" "9'):
        a['Status Projeto D09'] = a.pop('Status Projeto D" "9')
    if a.get('Nº Projeto D" "9'):
        a['Nº Projeto D09'] = a.pop('Nº Projeto D" "9')
    if a.get('Status Projeto D09') == 'Em análise / aprovação':
        a['Status Projeto D09'] = 'Em Análise/Aprovação'
    if a.get('Status Projeto D09') == 'Pendente elaborar':
        a['Status Projeto D09'] = a['Status Projeto D09'].title()
    if a['Priorização?'] is None or a['Priorização?'] == " ":
        a['Priorização?'] = "Não"
    if a['Classificação'] == 'Não comercial':
        a['Classificação'] = 'Não Comercial'
    if a['Cidade'] == 'São carlos':
        a['Cidade'] = 'São Carlos'
    if a['Cidade'] == 'Patrocinio Paulista':
        a['Cidade'] = 'Patrocínio Paulista'
    if a['Tipo Do Acesso'] == 'Compatilhado':
        a['Tipo Do Acesso'] = 'Compartilhado'
    else:
        continue

In [30]:
# point, road = get_road_coordinates(values[144]['Rodovia'], values[144]['km'], str(get_connected_reference(company, 'reporting', 'direction', values[144]['Sentido'].capitalize())) , company)
# print(values[144]['Rodovia'],values[143]['Rodovia'],road)

In [25]:
objects=[]

for index, a in enumerate(tqdm(values)):

    point, road = get_road_coordinates(a['Rodovia'], a['km'], str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'].capitalize())) , company)
    if road == None:
        road_manipulada = a['Rodovia'].split(' - ')[0].replace(' ','-')
        print(road_manipulada)
        point, road = get_road_coordinates(road_manipulada, a['km'], str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'].capitalize())) , company)
#     print(road.name)
    try:
        objects.append(Reporting(
                    company=company,
                    firm=equipe,
                    occurrence_type=occurrence_type,
                    lane=get_connected_reference(company, 'reporting', 'lane', 'Faixa 1'),
                    road=road,
                    road_name=road.name,
                    direction=get_connected_reference(company, 'reporting', 'direction', a['Sentido'].capitalize()),
                    created_by=user,
                    status=status,
#                     found_at=datetime.datetime(2023, 5, 16, 0, 0).replace(tzinfo=utc),
                    km=a['km'],
#                     end_km=a['km final'],
                    point=point,
                    form_data={
                        'cod_novo': a['Código Novo'],
                        'cod_acesso': a['Código Do Acesso'],
                        'lote': a['Lote'],
                        'tipo_da_obra': a['Tipo Da Obra'],
                        'localization': a['Localização'] if a['Localização'] is not None else None,
                        'latitude': a['Latitude'],
                        'longitude': a['Longitude'],
                        'city': get_value('Cidade',occurrence_type,a['Cidade'].strip()) if 'Cidade' in a else None,
                        'interessado': a['Interessado'],
                        'matricula': str(a['Matrícula']) if a['Matrícula'] is not '-' else None,
                        'cri': str(a['Cri']) if type(a['Cri']) is str and a['Cri'] != '-' else None,
                        'tipo_acesso': get_value('Tipo Do Acesso',occurrence_type,a['Tipo Do Acesso']) if 'Tipo Do Acesso' in a else None,
                        'classificacao': get_value('Classificação',occurrence_type,a['Classificação']) if 'Classificação' in a else None,
                        'atividade_do_acesso': get_value('Atividade Do Acesso',occurrence_type,a['Atividade Do Acesso']) if 'Atividade Do Acesso' in a else None,
                        'atividade': get_value('Atividade',occurrence_type,a['Atividade'].rstrip()) if 'Atividade' in a and a['Atividade'] is not None and a['Atividade'] != ' ' else None,
                        'autorizado': True if a['Autorizado?'].lower() == "sim" else False,
                        'n_protocolo_artesp': str(a['Nº Protocolo Artesp']),
                        'n_processo_artesp': str(a['Nº Processo Artesp']),
                        'termo_autorizacao': a['Termo De Autorização'],
                        'expediente': a['Expediente'],
                        'executado_conforme': True if a['Executado Conforme Projeto Aprovado E Autorizado?'].lower() == "sim" else False,
                        'atende_normas': True if a['Atende As Normas Atuais?'].lower() == "sim" else False,
                        'estado_conserva': get_value('Estado De Conservação',occurrence_type,a['Estado De Conservação'].capitalize().strip()) if 'Estado De Conservação' in a and a['Estado De Conservação'].lower() != 'n/a' else None,
                        'n_projeto': a['Nº Projeto D09'],
                        'status_projeto': get_value('Status Projeto D09',occurrence_type,a['Status Projeto D09'])if a['Status Projeto D09'] and a['Status Projeto D09'] != ' ' else None,
                        'procedimento': get_value('Procedimento',occurrence_type,a['Procedimento']) if 'Procedimento' in a and a['Procedimento'] != ' ' else None,
                        'solucao': a['Solução'],
                        'priorizacao': True if a['Priorização?'].lower() == "sim" else False,
                        'contato': a['Contato'],
                        'telefone': a['Telefone'],
                        'email': a['E-Mail'],
                        'endereco': a['Endereço Para Envio De Correspondência'],
                        'notificado': True if a['Notificado?'].lower() == "sim" else False,
                        'notification': a['Notificacao'],
                        'cience': a['Termo Ciencia'],
                        'anuence': a['Termo Anuencia'],
                        'notes': a['Observações'],
                        },
                    ))
    except Exception as e:
#         exception_type, exception_object, exception_traceback = sys.exc_info()
#         line_number = exception_traceback.tb_lineno
#         if a["Rodovia"] == 'SP-310':
#             print(f'AQUI: Km não existe no sistema: {a["km"]} | Rodovia não existe no sistema: {a["Rodovia"]} | Linha da Planilha: {index+2} | {road} | {road_manipulada}')
#             continue
        print(f'AQUI: Km: {a["km"]} | Rodovia: {a["Rodovia"]} | Linha da Planilha: {index+2} erro: {e}')
        print(objects.form_data)
        pass

  0%|          | 0/232 [00:00<?, ?it/s]

AttributeError: 'NoneType' object has no attribute 'capitalize'

In [26]:
objects[103].__dict__

{'_state': <django.db.models.base.ModelState at 0x7f66f225aa50>,
 'uuid': UUID('df2a0142-869d-4d20-b5a2-0a905c490d98'),
 'number': '',
 'company_id': UUID('ec364a8e-b335-4729-8ac8-50ccabd73459'),
 'road_name': 'SP-330',
 'road_id': 1926,
 'km': 317.704,
 'end_km': None,
 'km_reference': None,
 'project_km': 0,
 'project_end_km': None,
 'point': <Point object at 0x7f66f1fa5490>,
 'direction': '0',
 'lane': '1',
 'track': None,
 'branch': None,
 'address': {},
 'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
 'firm_id': UUID('5d9b180c-22d3-4e8a-a887-de3cfe7c4cc9'),
 'occurrence_type_id': UUID('7c44852b-84d8-453f-bfc9-d3e2e92602b2'),
 'form_data': {'cod_novo': '029-N-SP330-317-N02',
  'lote': 29,
  'tipo_da_obra': 'Implantação da Faixa Adicional do km 304+000 ao km 318+500',
  'localization': 317.704,
  'latitude': -21.117926,
  'longitude': -47.774127,
  'city': '19',
  'interessado': 'Reserva Regatas Loteamento LTDA - Lemos Palma Agricola e Empreendimentos LTDA',
  'matri

In [27]:
for reporting in tqdm(objects):
    reporting.save()
    print(reporting.number)

  0%|          | 0/104 [00:00<?, ?it/s]

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.executed_at received a naive datetime (2023-08-23 15:37:03.677864) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalReporting.executed_at received a naive datetime (2023-08-23 15:37:03.677864) while time zone support is active.
  RuntimeWarning)


Searching KM on direction...
Found KM on Road 1926: SP-330
VPT-IT-2023.03081
Searching KM on direction...
Found KM on Road 1926: SP-330
VPT-IT-2023.03082


/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.executed_at received a naive datetime (2023-08-23 15:37:04.354641) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalReporting.executed_at received a naive datetime (2023-08-23 15:37:04.354641) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.executed_at received a naive datetime (2023-08-23 15:37:04.515968) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:

Searching KM on direction...
Searching KM without direction...
Found KM on Road 1926: SP-330
VPT-IT-2023.03083
Searching KM on direction...
Found KM on Road 1926: SP-330


...

Searching KM on direction...
Found KM on Road 1926: SP-330
VPT-IT-2023.03182


/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.executed_at received a naive datetime (2023-08-23 15:37:31.733239) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalReporting.executed_at received a naive datetime (2023-08-23 15:37:31.733239) while time zone support is active.
  RuntimeWarning)


Searching KM on direction...
Searching KM without direction...
Found KM on Road 1926: SP-330
VPT-IT-2023.03183


/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.executed_at received a naive datetime (2023-08-23 15:37:32.088019) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalReporting.executed_at received a naive datetime (2023-08-23 15:37:32.088019) while time zone support is active.
  RuntimeWarning)


Searching KM on direction...
Searching KM without direction...
Found KM on Road 1926: SP-330
VPT-IT-2023.03184


In [43]:
# for reporting, images in tqdm(objects):
#     for image in reversed(images):
#         reporting_file = ReportingFile(
#             created_by=user,
#             reporting=reporting,
#             description=image.name,
#             km=reporting.km,
#             point=reporting.point
#         )
#         reporting_file.save()
#         image_io = BytesIO(image._data())
#         im = Image.open(image_io)
#         if im.mode in ("RGBA", "P"):
#             im = im.convert("RGB")
#         thumb_io = BytesIO()
#         im.save(thumb_io, format='jpeg', quality=90)
#         reporting_file.upload.save(image.name + '.jpeg', thumb_io)

  0%|          | 0/585 [00:00<?, ?it/s]

In [ ]:
rep=Reporting.objects.filter(occurrence_type__name="Acessos",road__name='SP-330',company__name='Arteris Via Paulista - Faixa de Domínio')

In [20]:
for a in tqdm(values):
    try:
        rep=Reporting.objects.get(
            occurrence_type__name="Acessos",                 
            road__name='SP-330',
            company__name='Arteris Via Paulista - Faixa de Domínio',
            form_data__cod_novo=a['Código Novo']
        )
        rep.form_data['cod_acesso'] = a['Código Do Acesso']
        rep.save()
    except:
        print(a['Código Novo'])

  0%|          | 0/232 [00:00<?, ?it/s]

Searching KM on direction...
Found KM on Road 1926: SP-330
Searching KM on direction...
Found KM on Road 1926: SP-330
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1926: SP-330
Searching KM on direction...
Found KM on Road 1926: SP-330
Searching KM on direction...
Found KM on Road 1926: SP-330
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1926: SP-330
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1926: SP-330
Searching KM on direction...
Found KM on Road 1926: SP-330
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1926: SP-330
Searching KM on direction...
Found KM on Road 1926: SP-330
Searching KM on direction...
Found KM on Road 1926: SP-330
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1926: SP-330
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1926: SP-330
Searching KM on direction...


DoesNotExist: Reporting matching query does not exist.

In [2]:
reps=Reporting.objects.filter(
        occurrence_type__name="Acessos",                 
        road__name__in=['SP-281'],
        company__name='Arteris Via Paulista - Faixa de Domínio'
    )
reps.count()

145

In [3]:
for a in reps:
    a.form_data['lote']=29
    a.save()

Searching KM on direction...
Found KM on Road 1921: SP-281
Searching KM on direction...
Found KM on Road 1921: SP-281
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1921: SP-281
Searching KM on direction...
Found KM on Road 1921: SP-281
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1921: SP-281
Searching KM on direction...
Found KM on Road 1921: SP-281
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1921: SP-281
Searching KM on direction...
Found KM on Road 1921: SP-281
Searching KM on direction...
Found KM on Road 1921: SP-281
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1921: SP-281
Searching KM on direction...
Found KM on Road 1921: SP-281
Searching KM on direction...
Found KM on Road 1921: SP-281
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1921: SP-281
Searching KM on direction...
Searching KM without direction...


Searching KM on direction...
Found KM on Road 1921: SP-281
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1921: SP-281
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1921: SP-281
Searching KM on direction...
Found KM on Road 1921: SP-281
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1921: SP-281
Searching KM on direction...
Found KM on Road 1921: SP-281
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1921: SP-281
Searching KM on direction...
Found KM on Road 1921: SP-281
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1921: SP-281
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1921: SP-281
Searching KM on direction...
Found KM on Road 1921: SP-281
Searching KM on direction...
Found KM on Road 1921: SP-281
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1921: SP